In [18]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///new.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [19]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso 

from sklearn.metrics import root_mean_squared_error

In [20]:
import mlflow
mlflow.set_tracking_uri("sqlite:///new.db")
mlflow.set_experiment("nyc-taxi-experiment")

2024/06/09 19:53:20 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/kamilakolpashnikova/Dropbox/Mac/Documents/Jupyter_notebooks/mlops-2024/02-experiment-tracking/mlruns/1', creation_time=1717977200591, experiment_id='1', last_update_time=1717977200591, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [21]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']

    df['duration'] = df['duration'].dt.total_seconds() / 60
    
    df = df[(df.duration>=1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [22]:
df_train = read_dataframe('data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('data/yellow_tripdata_2023-02.parquet')
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
5,1,2023-02-01 00:52:40,2023-02-01 01:07:18,1.0,5.10,1.0,N,148,236,1,21.9,3.50,0.5,5.35,0.0,1.0,32.25,2.5,0.00,14.633333
6,1,2023-02-01 00:12:39,2023-02-01 00:40:36,1.0,8.90,1.0,N,137,244,1,41.5,3.50,0.5,3.50,0.0,1.0,50.00,2.5,0.00,27.950000


In [23]:
train_dict = df_train[['PULocationID', 'DOLocationID']].to_dict(orient='records')
val_dict = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [24]:
# Create the DictVectorizer
dv = DictVectorizer()

# Fit and transform the data
X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

In [25]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [26]:
alpha = 0.01

lr = Lasso(alpha)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

8.138118733385994

In [31]:
with mlflow.start_run():
    mlflow.set_tag("developer","Qfl3x")
    
    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("val-data-path", "data/green_tripdata_2021-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

In [32]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [12]:


train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100, # was 1000 in the original code
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:06:33] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.85020                           
[1]	validation-rmse:7.33480                           
[2]	validation-rmse:6.71042                           
[3]	validation-rmse:6.58682                           
[4]	validation-rmse:6.50465                           
[5]	validation-rmse:6.41720                           
[6]	validation-rmse:6.15263                           
[7]	validation-rmse:6.08645                           
[8]	validation-rmse:6.03566                           
[9]	validation-rmse:5.97975                           
[10]	validation-rmse:5.90972                          
[11]	validation-rmse:5.78353                          
[12]	validation-rmse:5.76575                          
[13]	validation-rmse:5.72657                          
[14]	validation-rmse:5.65600                          
[15]	validation-rmse:5.56602                          
[16]	validation-rmse:5.55760                          
[17]	validation-rmse:5.51289                          
[18]	valid

/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:06:54] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:8.80715                                                    
[3]	validation-rmse:8.59567                                                    
[4]	validation-rmse:8.44180                                                    
[5]	validation-rmse:8.33175                                                    
[6]	validation-rmse:8.23966                                                    
[7]	validation-rmse:8.17887                                                    
[8]	validation-rmse:8.12912                                                    
[9]	validation-rmse:8.09004                                                    
[10]	validation-rmse:8.04343                                                   
[11]	validation-rmse:8.01296                                                   
[12]	validation-rmse:7.98865                                                   
[13]	validation-rmse:7.96442                                                   
[14]	validation-rmse:7.94137            

...

[0]	validation-rmse:8.86477                                                     
[1]	validation-rmse:8.11219                                                     
[2]	validation-rmse:7.26710                                                     
[3]	validation-rmse:6.98899                                                     
[4]	validation-rmse:6.81580                                                     
[5]	validation-rmse:6.40913                                                     
[6]	validation-rmse:6.33237                                                     
[7]	validation-rmse:6.27599                                                     
[8]	validation-rmse:6.21028                                                     
[9]	validation-rmse:6.17376                                                     
[10]	validation-rmse:5.93359                                                    
[11]	validation-rmse:5.91376                                                    
[12]	validation-rmse:5.88888

/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:35:02] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.67791                                                     
[1]	validation-rmse:9.33418                                                     
[2]	validation-rmse:9.03176                                                     
[3]	validation-rmse:8.72879                                                     
[4]	validation-rmse:8.49788                                                     
[5]	validation-rmse:8.22660                                                     
[6]	validation-rmse:8.03923                                                     
[7]	validation-rmse:7.75564                                                     
[8]	validation-rmse:7.50845                                                     
[9]	validation-rmse:7.38129                                                     
[10]	validation-rmse:7.17316                                                    
[11]	validation-rmse:7.07565                                                    
[12]	validation-rmse:6.90754

/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:35:38] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.75988                                                     
[1]	validation-rmse:6.11451                                                     
[2]	validation-rmse:5.83648                                                     
[3]	validation-rmse:5.46542                                                     
[4]	validation-rmse:5.42017                                                     
[5]	validation-rmse:5.32868                                                     
[6]	validation-rmse:5.31280                                                     
[7]	validation-rmse:5.29884                                                     
[8]	validation-rmse:5.26016                                                     
[9]	validation-rmse:5.25629                                                     
[10]	validation-rmse:5.24054                                                    
[11]	validation-rmse:5.23755                                                    
[12]	validation-rmse:5.23193

In [33]:
params = {
    'learning_rate': 0.19408972548566633,
    'max_depth': 100,
    'min_child_weight': 1.8824847254189034,
    'objective': 'reg:linear',
    'reg_alpha': 0.13078016502323883,
    'reg_lambda': 0.0001070736826826826,
    'seed': 42
}

mlflow.xgboost.autolog()


booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=100, # was 1000 in the original code
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2024/06/09 19:56:48 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1e0774bee771491b8f0065a11743b93a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:56:50] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.98747
[1]	validation-rmse:8.25230
[2]	validation-rmse:7.73872
[3]	validation-rmse:7.03330
[4]	validation-rmse:6.52753
[5]	validation-rmse:6.25668
[6]	validation-rmse:6.11893
[7]	validation-rmse:5.85720
[8]	validation-rmse:5.78975
[9]	validation-rmse:5.62370
[10]	validation-rmse:5.53003
[11]	validation-rmse:5.50327
[12]	validation-rmse:5.43364
[13]	validation-rmse:5.41354
[14]	validation-rmse:5.40036
[15]	validation-rmse:5.38914
[16]	validation-rmse:5.33734
[17]	validation-rmse:5.32942
[18]	validation-rmse:5.29641
[19]	validation-rmse:5.29214
[20]	validation-rmse:5.28834
[21]	validation-rmse:5.28227
[22]	validation-rmse:5.25889
[23]	validation-rmse:5.25669
[24]	validation-rmse:5.25386
[25]	validation-rmse:5.23956
[26]	validation-rmse:5.23769
[27]	validation-rmse:5.23330
[28]	validation-rmse:5.23002
[29]	validation-rmse:5.22853
[30]	validation-rmse:5.22743
[31]	validation-rmse:5.21863
[32]	validation-rmse:5.21784
[33]	validation-rmse:5.21728
[34]	validation-rmse:5.2

2024/06/09 19:57:23 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/06/09 19:57:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:57:23] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
2024/06/09 19:57:24 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This m

In [34]:
mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:58:17] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [35]:
mlflow.xgboost.autolog(disable=True)

In [38]:
import pickle

with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.19408972548566633,
        'max_depth': 100,
        'min_child_weight': 1.8824847254189034,
        'objective': 'reg:linear',
        'reg_alpha': 0.13078016502323883,
        'reg_lambda': 0.0001070736826826826,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100, # was 1000 in the original code
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")



/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:08:20] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.98747
[1]	validation-rmse:8.25230
[2]	validation-rmse:7.73872
[3]	validation-rmse:7.03330
[4]	validation-rmse:6.52753
[5]	validation-rmse:6.25668
[6]	validation-rmse:6.11893
[7]	validation-rmse:5.85720
[8]	validation-rmse:5.78975
[9]	validation-rmse:5.62370
[10]	validation-rmse:5.53003
[11]	validation-rmse:5.50327
[12]	validation-rmse:5.43364
[13]	validation-rmse:5.41354
[14]	validation-rmse:5.40036
[15]	validation-rmse:5.38914
[16]	validation-rmse:5.33734
[17]	validation-rmse:5.32942
[18]	validation-rmse:5.29641
[19]	validation-rmse:5.29214
[20]	validation-rmse:5.28834
[21]	validation-rmse:5.28227
[22]	validation-rmse:5.25889
[23]	validation-rmse:5.25669
[24]	validation-rmse:5.25386
[25]	validation-rmse:5.23956
[26]	validation-rmse:5.23769
[27]	validation-rmse:5.23330
[28]	validation-rmse:5.23002
[29]	validation-rmse:5.22853
[30]	validation-rmse:5.22743
[31]	validation-rmse:5.21863
[32]	validation-rmse:5.21784
[33]	validation-rmse:5.21728
[34]	validation-rmse:5.2

/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:09:12] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/_distutils_hack/_

In [39]:
with mlflow.end_run():
    pass

AttributeError: __enter__

In [40]:
logged_model = "runs:/f30801391f284e0aaccfd60cba9a7aec/models_mlflow"

loaded_model = mlflow.pyfunc.load_model(logged_model)

/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:11:08] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [41]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: f30801391f284e0aaccfd60cba9a7aec

In [42]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

/Users/kamilakolpashnikova/anaconda3/envs/exp_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:12:21] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [43]:
xgboost_model

In [46]:
y_pred = xgboost_model.predict(valid)

In [47]:
y_pred

array([ 6.3495045, 44.144955 , 15.8901   , ..., 15.219723 , 11.963885 ,
       13.507035 ], dtype=float32)

In [48]:
## creating new experiment

client.create_experiment("some-new-experiments-2")

'2'

In [53]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1", 
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=2,
    order_by=["metrics.rmse ASC"]
    )


In [54]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f30801391f284e0aaccfd60cba9a7aec, rmse: 5.1910
run id: 25efc624f1ac45b68366653dd21dfbe4, rmse: 8.1381


In [55]:
run_id = "25efc624f1ac45b68366653dd21dfbe4"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri, "nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1717979370024, current_stage='None', description=None, last_updated_timestamp=1717979370024, name='nyc-taxi-regressor', run_id='25efc624f1ac45b68366653dd21dfbe4', run_link=None, source='/Users/kamilakolpashnikova/Dropbox/Mac/Documents/Jupyter_notebooks/mlops-2024/02-experiment-tracking/mlruns/1/25efc624f1ac45b68366653dd21dfbe4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [58]:
client.get_registered_model("nyc-taxi-regressor")

<RegisteredModel: aliases={}, creation_timestamp=1717978511344, description='', last_updated_timestamp=1717979370024, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1717979370024, current_stage='None', description=None, last_updated_timestamp=1717979370024, name='nyc-taxi-regressor', run_id='25efc624f1ac45b68366653dd21dfbe4', run_link=None, source='/Users/kamilakolpashnikova/Dropbox/Mac/Documents/Jupyter_notebooks/mlops-2024/02-experiment-tracking/mlruns/1/25efc624f1ac45b68366653dd21dfbe4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>

In [62]:
latest_versions = client.get_latest_versions("nyc-taxi-regressor")

/var/folders/rx/2p72xf417cn83_mvjvthxc5h0000gn/T/ipykernel_33446/2816607893.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions("nyc-taxi-regressor")


In [63]:
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None


In [65]:
client.transition_model_version_stage(
    name="nyc-taxi-regressor",
    version=1,
    stage="Staging",
    archive_existing_versions=False
)

/var/folders/rx/2p72xf417cn83_mvjvthxc5h0000gn/T/ipykernel_33446/3351308404.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1717978511370, current_stage='Staging', description='', last_updated_timestamp=1717979813694, name='nyc-taxi-regressor', run_id='f30801391f284e0aaccfd60cba9a7aec', run_link='', source='/Users/kamilakolpashnikova/Dropbox/Mac/Documents/Jupyter_notebooks/mlops-2024/02-experiment-tracking/mlruns/1/f30801391f284e0aaccfd60cba9a7aec/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [66]:
from datetime import datetime

date = datetime.today().date().isoformat()

model_version = 1

client.update_model_version(
    name="nyc-taxi-regressor",
    version=model_version,
    description=f"Model version {model_version} trained on NYC Taxi dataset on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1717978511370, current_stage='Staging', description='Model version 1 trained on NYC Taxi dataset on 2024-06-09', last_updated_timestamp=1717980052577, name='nyc-taxi-regressor', run_id='f30801391f284e0aaccfd60cba9a7aec', run_link='', source='/Users/kamilakolpashnikova/Dropbox/Mac/Documents/Jupyter_notebooks/mlops-2024/02-experiment-tracking/mlruns/1/f30801391f284e0aaccfd60cba9a7aec/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>

def preprocess(df, dv):
    

In [67]:
def preprocess(df, dv):
    train_dict = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
    return dv.transform(train_dict)

def test_model(model_uri, X_test, y_test):
    model = mlflow.pyfunc.load_model(model_uri)
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

Q1

Q2 4

Q3 min_samples_split 2

Q4 default-artifact-root

Q5 5.335

Q6 test_rmse
5.567408012462019